# Male or Female?

In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
from matplotlib import pyplot as plt

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import feature_extraction, pipeline, cross_validation

import statsmodels.api as sm
from sklearn import linear_model
from sklearn import metrics

from IPython.display import Image
from IPython.display import display
import seaborn as sns

import re


In [2]:
df = pd.read_excel('/Users/JHYL/DSI-HK-1/classes/week-04/5.2-lab/assets/bloggender.xls')
pd.set_option('max_colwidth', 100000)
df.head()

text  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [3]:
df['gender'].unique()

array([u'M', u'F', u' M', u' F', u'F ', u' M ', u'm', u'f'], dtype=object)

In [4]:
df.gender = df.gender.str.replace(' ', '')
df.gender = df.gender.str.lower()
#([' M', ' F', 'F ', ' M', 'm', 'f'], ['M', 'F', 'F', 'M', 'M', 'F'])
df.to_excel('blog_gender_clean.xls')

In [5]:
df.gender.unique()

array([u'm', u'f'], dtype=object)

In [6]:
df.dropna(inplace=True)

In [7]:
X = df.text
y = df.gender

In [8]:
text_pipeline = pipeline.make_pipeline(feature_extraction.text.CountVectorizer(),
                                       linear_model.LogisticRegression())

In [9]:
cv = cross_validation.KFold(len(y), n_folds=5, shuffle=True)
print cv

sklearn.cross_validation.KFold(n=3226, n_folds=5, shuffle=True, random_state=None)


In [10]:
cross_validation.cross_val_score(text_pipeline, X=X, y=y, cv=cv)

array([ 0.65170279,  0.67286822,  0.69922481,  0.66511628,  0.68682171])

In [11]:
X_test = ['I love unicorns']

In [12]:
text_pipeline.fit(X=X, y=y)

Pipeline(steps=[('countvectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [13]:
def m_f(text):
    g_predict = text_pipeline.predict(text)
    g_prob = text_pipeline.predict_proba(text)
    return g_predict, g_prob

In [14]:
print m_f(X_test)

(array([u'f'], dtype=object), array([[ 0.54310341,  0.45689659]]))


# TFIDF Vectorizer

In [20]:
text_pipeline_tfidf = pipeline.make_pipeline(feature_extraction.text.TfidfVectorizer(),
                                       linear_model.LogisticRegression())

In [21]:
cv_tfidf = cross_validation.KFold(len(y), n_folds=5, shuffle=True)
print cv_tfidf

sklearn.cross_validation.KFold(n=3226, n_folds=5, shuffle=True, random_state=None)


In [22]:
cross_validation.cross_val_score(text_pipeline_tfidf, X=X, y=y, cv=cv)

array([ 0.68421053,  0.71627907,  0.69147287,  0.6744186 ,  0.69147287])

In [108]:
X_test = ['dude']

In [109]:
text_pipeline_tfidf.fit(X=X, y=y)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smoo...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [110]:
def m_f_tfidf(text):
    g_predict_tfidf = text_pipeline_tfidf.predict(text)
    g_prob_tfidf = text_pipeline_tfidf.predict_proba(text)
    return g_predict_tfidf, g_prob_tfidf

In [111]:
print m_f_tfidf(X_test)

(array([u'm'], dtype=object), array([[ 0.3598101,  0.6401899]]))


# Grid Search

In [43]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.grid_search import GridSearchCV
from sklearn import linear_model, preprocessing, cross_validation, metrics, pipeline, grid_search

In [32]:
X = df.text
y = df.gender

In [33]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.5, stratify=y, random_state=77)

In [38]:
# build confusion matrices for the models above
Y_1_pred = text_pipeline.predict(X_train)

conmat_1 = confusion_matrix(Y_train, Y_1_pred)
conmat_1 = pd.DataFrame(conmat_1)

print(conmat_1)

     0    1
0  661  113
1   86  753


In [66]:
param_grid = {
    'tfidfvectorizer__ngram_range': [(1,1), (1,2), (2,2)],
    'tfidfvectorizer__lowercase': [True, False],
    'logisticregression__fit_intercept': [True, False],
    'logisticregression__penalty': ['l1', 'l2']    
}

gs = grid_search.GridSearchCV(text_pipeline_tfidf, param_grid=param_grid, cv=3, verbose=1, n_jobs=1)

In [67]:
gs

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smoo...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'tfidfvectorizer__ngram_range': [(1, 1), (1, 2), (2, 2)], 'tfidfvectorizer__lowercase': [True, False], 'logisticregression__penalty': ['l1', 'l2'], 'logisticregression__fit_intercept': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [73]:
gs.fit(X,y)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:  5.7min
[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed:  8.4min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smoo...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'tfidfvectorizer__ngram_range': [(1, 1), (1, 2), (2, 2)], 'tfidfvectorizer__lowercase': [True, False], 'logisticregression__penalty': ['l1', 'l2'], 'logisticregression__fit_intercept': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [74]:
gs.best_estimator_

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smo...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [75]:
gs.best_params_

{'logisticregression__fit_intercept': False,
 'logisticregression__penalty': 'l2',
 'tfidfvectorizer__lowercase': False,
 'tfidfvectorizer__ngram_range': (1, 1)}

In [76]:
gs.best_score_

0.69373837569745811

In [102]:
X_test = ['my favourite colour is pink']
gs.predict(X_test)

array([u'f'], dtype=object)

In [133]:
from ipywidgets import widgets
from IPython.display import display
text = widgets.Text()
display(text)

def handle_submit(sender):
    print [text.value], gs.predict([text.value])

text.on_submit(handle_submit)

[u'Cleveland (CNN)Donald Trump conjured a dire picture Thursday of an America sliding deeper into poverty, violence and corruption and declared himself the only person who could avert disaster. Accepting the Republican nomination in Cleveland, the billionaire twice pledged to be a "voice" for working Americans, restore law and order and to confound elites and doubters by winning the White House in November.'] [u'm']
[u'Cleveland (CNN)Donald Trump conjured a dire picture Thursday of an America sliding deeper into poverty, violence and corruption and declared himself the only person who could avert disaster. Accepting the Republican nomination in Cleveland, the billionaire twice pledged to be a "voice" for working Americans, restore law and order and to confound elites and doubters by winning the White House in November.'] [u'm']
[u'i love unicorns'] [u'f']
[u'dude'] [u'm']
[u'beer'] [u'm']
[u'babe'] [u'f']
[u'pizza'] [u'm']
[u'data science'] [u'm']
[u'general assembly'] [u'm']
